In [20]:
using DiffEqSensitivity, OrdinaryDiffEq, DiffEqParamEstim,
      ParameterizedFunctions, BlackBoxOptim, NLopt, Calculus, ForwardDiff, Optim, BenchmarkTools, RecursiveArrayTools

In [21]:
include("/Users/vaibhav/DiffEqParamEstim.jl/src/localsaparamestim.jl")

costfunc_gradient (generic function with 1 method)

In [22]:
fitz  =  @ode_def FitzhughNagumo begin
  dv = v - v^3/3 -w + 0.5
  dw = -tinv*(v +  a - 0.8*w)
end a tinv

ErrorException: invalid redefinition of constant FitzhughNagumo

In [23]:
p  =  [0.7,0.08]                      # Parameters used to construct the dataset
r0 = [1.0; 1.0]                     # initial value
tspan = (0.0, 3.0)                 # sample of 3000 observations over the (0,30) timespan
prob = ODEProblem(fitz, r0, tspan,p)

ODEProblem with uType Array{Float64,1} and tType Float64. In-place: true
timespan: (0.0, 3.0)
u0: [1.0, 1.0]

In [24]:
global t = collect(range(0, stop=3.0, length=1000))
data = Array(solve(prob,Tsit5(),saveat=t,abstol=1e-3,reltol=1e-3))
ourprob = ODELocalSensitivityProblem(fitz,r0,tspan,p)
oursol = solve(ourprob,Tsit5(),saveat=t,abstol=1e-6,reltol=1e-6)

retcode: Success
Interpolation: 1st order linear
t: 1000-element Array{Float64,1}:
 0.0                 
 0.003003003003003003
 0.006006006006006006
 0.009009009009009009
 0.012012012012012012
 0.015015015015015015
 0.018018018018018018
 0.021021021021021023
 0.024024024024024024
 0.02702702702702703 
 0.03003003003003003 
 0.03303303303303303 
 0.036036036036036036
 ⋮                   
 2.966966966966967   
 2.96996996996997    
 2.972972972972973   
 2.975975975975976   
 2.978978978978979   
 2.981981981981982   
 2.984984984984985   
 2.987987987987988   
 2.990990990990991   
 2.993993993993994   
 2.996996996996997   
 3.0                 
u: 1000-element Array{Array{Float64,1},1}:
 [1.0, 1.0, 0.0, 0.0, 0.0, 0.0]                                       
 [1.0005, 0.999784, 3.15651e-6, -0.0021023, 4.05938e-6, -0.00270397]  
 [1.001, 0.999567, 1.26268e-5, -0.00420501, 1.62426e-5, -0.0054105]   
 [1.0015, 0.999351, 2.84121e-5, -0.00630813, 3.65572e-5, -0.00811957] 
 [1.00201, 0.99913

In [6]:
oursol[3,100]

0.030906400389873762

In [7]:
out = Array{Float64}(undef,2)
p = [0.7,0.01]
costfunc_gradient(p,out)

30.7133361592769

In [8]:
out

2-element Array{Float64,1}:
 -534.6017489903944
 -815.6459048635078

In [9]:
out = Array{Float64}(undef,2)

2-element Array{Float64,1}:
 8.4879831663e-314
 2.121995792e-314 

In [10]:
Calculus.finite_difference!(costfunc,p,out,:central)

In [11]:
out

2-element Array{Float64,1}:
   -7.637167444852697
 -815.6458940224957  

In [12]:
p = [0.5,0.01]
@benchmark begin
    out = Array{Float64}(undef,2)
    costfunc_gradient(p,out)
end

BenchmarkTools.Trial: 
  memory estimate:  638.72 KiB
  allocs estimate:  3552
  --------------
  minimum time:     551.461 μs (0.00% GC)
  median time:      636.817 μs (0.00% GC)
  mean time:        786.835 μs (7.59% GC)
  maximum time:     6.872 ms (78.78% GC)
  --------------
  samples:          6292
  evals/sample:     1

In [13]:
out = Array{Float64}(undef,2)
costfunc_gradient(p,out)

32.261072788753445

In [14]:
out

2-element Array{Float64,1}:
 -392.01858732469236
 -678.1275946016888 

In [15]:
out = Array{Float64}(undef,2)

2-element Array{Float64,1}:
 2.1540038694e-314
 2.154020462e-314 

In [16]:
@benchmark Calculus.finite_difference!(costfunc,p,out,:central)

BenchmarkTools.Trial: 
  memory estimate:  1.19 MiB
  allocs estimate:  7106
  --------------
  minimum time:     1.061 ms (0.00% GC)
  median time:      1.145 ms (0.00% GC)
  mean time:        1.250 ms (7.56% GC)
  maximum time:     4.443 ms (52.30% GC)
  --------------
  samples:          3993
  evals/sample:     1

In [17]:
out

2-element Array{Float64,1}:
   -7.840372797181787
 -678.127600568777   

In [18]:
opt = Opt(:LD_MMA, 2)
min_objective!(opt, costfunc_gradient)
lower_bounds!(opt,[0.0,0.0])
upper_bounds!(opt,[1.0,0.1])
xtol_rel!(opt,1e-12)
maxeval!(opt, 100000)
(minf,minx,ret) = NLopt.optimize(opt,[0.5,0.0])

(0.006484500222514104, [0.927343, 0.0665975], :XTOL_REACHED)